<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="IMG/logo_IGP.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Assignment 02</h1>
<h1>Modelado Numérico de la Atmósfera</h1>
        <p>Carlos Enciso Ojeda<br/>
        <a href="https://github.com/carlosenciso/WRF_IGP/">www.atmcenciso.com</a><br/>
            @carlos.enciso.o, senamhi@cenciso</p></div>
</div>

<h2><strong>Modules</strong></h2>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import integrate
import cmocean as cmo
import xarray as xr
import imageio
import os
plt.style.use('seaborn')

<h2><strong>Printing decorator</strong></h2>

In [2]:
def _decorator(func):
    def wrap(*args, **kwargs):
        print('#'+'-'*(len(*args)+2))
        func(*args)
        print('#'+'-'*(len(*args)+2))
    return wrap
@_decorator
def printing(msg):
        print('# '+msg)

<h2><strong>Parameters</strong></h2>

In [3]:
wind_speed = 8*1e-3
wind_direc = 272
dx = dy = 18
simulated_time = 2*3600
uwind = np.cos(np.deg2rad(wind_direc-270))*wind_speed
vwind = -np.sin(np.deg2rad(wind_direc-270))*wind_speed

<h2><strong>Reading Dataset</strong></h2>

In [4]:
Pres = pd.read_csv('Dataset/dataset01_matrix.csv', header=None)+1e3

## **Time step**

## $\frac{C*\Delta t}{\Delta r} \le 0.06$

In [5]:
tstep = lambda ws, wd, dx, dy: round(0.06*1/(ws*abs((np.cos(np.deg2rad(wd))))/dx + 
                                     ws*abs(np.sin(np.deg2rad(wd)))/dy),2)
time_step = tstep(wind_speed, wind_direc, dx, dy)
printing(f'Our time step must be less than {time_step} seconds')

#------------------------------------------------
# Our time step must be less than 130.52 seconds
#------------------------------------------------


## **Choosing** $\Delta t = 100s$

In [6]:
time_step = 100

In [7]:
N = round(simulated_time/time_step)+1
printing(f'Total N simulation is {N}')

#--------------------------
# Total N simulation is 73
#--------------------------


## **Total derivative for Pressure**

<h1>$$\frac{DP}{Dt} = \frac{\partial P}{\partial t} + u\frac{\partial P}{\partial x} + v\frac{\partial P}{\partial y} = 0$$</h1>

<h1>$$\frac{\partial P}{\partial t} = - u\frac{\partial P}{\partial x} - v\frac{\partial P}{\partial y}$$</h1>

<h1>$$P_{t+1} = P_{t} - [u\frac{\partial P}{\partial x} - v\frac{\partial P}{\partial y}]\Delta t$$</h1>

## **Create array 3D**

In [8]:
DPdt = np.array([Pres.values]*N)    

In [9]:
#---------------------
# Function Gradient
#---------------------
def eulerm(f,dx,dy,dt,u,v):
    dPdy,dPdx = np.gradient(f, dy,dx)
    F = f-dt*(u*dPdx+v*dPdy)
    return F

for n in range(N-1):
    DPdt[n+1,:,:] = eulerm(DPdt[n,:,:],dx,dy,time_step,uwind,vwind)

In [10]:
#------------------
# Mesh XY quiver
#------------------
X,Y = np.meshgrid(np.arange(.5,13.5), np.arange(.5,22.5))

In [31]:
%%capture
#------------------
# Plotting
#------------------
def advect_plot(n,N, vmin=None, vmax=None):
    fig, ax = plt.subplots(figsize=(13,12), dpi=80)
    pls = ax.contourf(DPdt[n,:,:], levels=np.linspace(vmin,vmax,60, endpoint=True), cmap=plt.cm.jet_r, 
                      zorder=1, vmin=vmin, vmax=vmax, origin='upper')
    CS  = ax.contour(DPdt[n,:,:], levels=17, zorder=2, linewidths=0.6, colors='black', 
                     vmin=vmax, vmax=vmin, origin='upper')
    ax.clabel(CS, CS.levels, inline=True, fontsize=8, fmt='%1.1f', colors='black')
    ax.grid(linestyle='--',color='gray',alpha=0.5)
    ax.set(xlabel='X axis', ylabel='Y axis') 
    ax.set_title(f'Time={N} seconds', loc='left')
    #ax.quiver(X,Y,np.ones_like(DPdt[0,:,:])*uwind,np.ones_like(DPdt[0,:,:])*vwind, headwidth=3, 
    #          linewidths=.2, zorder=1, alpha=.4)
    colorbar_ax = fig.add_axes([0.91, 0.125, 0.035, 0.755])
    fig.colorbar(pls, cax=colorbar_ax, aspect=15, label='Pressure (hPa)', ticks=np.arange(vmin,vmax+1,2))
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return image
kwargs_write = {'fps':4.0, 'quantizer':'nq'}
imageio.mimsave('FIGs/Presure_advect_HW2_CEO.gif', 
                [advect_plot(n, N, vmin=1017, vmax=1025) for n,N in enumerate(np.arange(0,simulated_time+1,time_step))], 
                fps=4)

In [32]:
![Alt Text](FIGs/Presure_advect_HW2_CEO.gif)

/bin/sh: 1: Syntax error: "(" unexpected
